In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda


llm = ChatOpenAI(
    temperature=0.1
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/allotment.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()


map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm

def map_docs(inputs):
    documents = inputs['documents']
    question = inputs['question']
    return "\n\n".join(
        map_doc_chain.invoke(
            {
                "context" : doc.page_content, "question" : question
            }
        ).content
        for doc in documents
    )

    # results = []
    # for document in documents:
    #     result = map_doc_chain.invoke({
    #         "context": document.page_content,
    #         "question": question
    #     }).content
    #     results.append(result)
    # return results

map_chain = { "documents": retriever, "question" : RunnablePassthrough() } | RunnableLambda(map_docs)
# {
#     "documents" : [Documents],
#     "question" : "What is the allotment meaning?"
# }

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", 
         """
         Given the following extracted parts of a long document and a question, create a final answer.
         If you don't know the answer, just say that you don't know. Don't try to make up an answer.
         ----
         {context} 
         """,
        #  this context is a final document convined every documents provided
        ),
        ("human", "{question}"),
    ]
)

chain = {"context": map_chain ,"question" : RunnablePassthrough() } | final_prompt | llm

chain.invoke("Who can use this garden and how much is it?")



AIMessage(content='The text does not specify who can use the garden or how much it costs. Allotment gardens are generally available for various people, and the cost is usually set below market prices to make it accessible. The land is typically owned by the municipality and rented to an allotment association, with each member of the association receiving a plot of land for gardening.')

In [ ]:
AIMessage(content='The text does not specify who can use the garden or how much it costs. Allotment gardens are generally available for various people, and the cost is usually set below market prices to make it accessible. The land is typically owned by the municipality and rented to an allotment association, with each member of the association receiving a plot of land for gardening.')